In [1]:
import sys
import os
import torch
from datetime import datetime

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

from Model.core import run_pipeline
from Model.data import DataModule
from Model.network import Model
from Model.trainer import Trainer
from Configs.config import config
from Utils.logger import Logger

In [2]:
project_root = current_dir  

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join(project_root, "runs", run_id)
checkpoint_dir = os.path.join(run_dir, "checkpoints")

model_save_path = os.path.join(checkpoint_dir, "best_model.pth")
log_dir = run_dir 

raw_path = os.path.join(project_root, config.paths.raw_data)
data_path = os.path.join(project_root, config.paths.train_data)

if not os.path.exists(data_path):
    os.makedirs(os.path.dirname(data_path), exist_ok=True)
    run_pipeline(raw_path, data_path)
    print(f"Generated new training data: {data_path}")

logger = Logger(log_dir=log_dir)

In [3]:
runs_dir = os.path.join(project_root, "runs")

print(f"Starting TensorBoard on {runs_dir}...")
%load_ext tensorboard
%tensorboard --logdir "{runs_dir}" --port 6014 --reload_interval 30

Starting TensorBoard on c:\Users\Victo\OneDrive\Área de Trabalho\Project-2\project - 2\runs...


Reusing TensorBoard on port 6014 (pid 9000), started 14:58:42 ago. (Use '!kill 9000' to kill it.)

In [4]:
try:
    dm = DataModule(
        data_path, 
        batch_size=config.model.batch_size, 
        num_workers=config.model.num_workers, 
        pin_memory=config.model.pin_memory, 
        logger=logger,
        use_balanced_sampling=config.model.use_balanced_sampling,
        oversample_ratio=config.model.oversample_ratio
    )
    dm.prepare_data()

    model = Model(
        embedding_dims=dm.emb_dims, 
        n_cont=len(dm.cont_cols), 
        outcome_dim=config.model.outcome_dim, 
        hidden_dim=config.model.hidden_dim, 
        n_blocks=config.model.n_blocks,     
        dropout=config.model.dropout, 
        n_heads=config.model.n_heads,
        use_grn=config.model.use_grn,
        use_cross_attention=config.model.use_cross_attention
    )
    
    logger.info(f"Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    trainer = Trainer(
        model, dm, 
        epochs=config.model.epochs, 
        lr=config.model.lr,
        weight_decay=config.model.weight_decay, 
        logger=logger,
        patience=config.model.patience, 
        mixed_precision=config.model.mixed_precision,
        checkpoint_dir=checkpoint_dir, 
        max_grad_norm=config.model.max_grad_norm,
        scheduler_factor=config.model.scheduler_factor,
        scheduler_patience=config.model.scheduler_patience, 
        min_lr=config.model.min_lr,
        loss_type=config.model.loss_type,
        focal_alpha=config.model.focal_alpha,
        focal_gamma=config.model.focal_gamma
    )
    
    best_model = trainer.fit()
    trainer.test(dm.test_dataloader())
    torch.save(best_model.state_dict(), model_save_path)
    print("Training finished.")

except Exception as e:
    logger.error(f"Failed: {e}")
    import traceback
    traceback.print_exc()
finally:
    logger.close()

2026-01-15 11:21:50,919 - logger - INFO - Loading data from c:\Users\Victo\OneDrive\Área de Trabalho\Project-2\project - 2\Data/training_data.csv
2026-01-15 11:21:51,309 - logger - INFO - Categorical Features: 9
2026-01-15 11:21:51,309 - logger - INFO - Continuous Features: 17
2026-01-15 11:21:51,523 - logger - INFO - Random Split Strategy Implemented (seed=42)
2026-01-15 11:21:51,527 - logger - INFO - Class imbalance ratio (neg:pos): 24.8:1
2026-01-15 11:21:51,551 - logger - INFO - Using balanced sampling with target ratio: 20.0%
2026-01-15 11:21:51,551 - logger - INFO - Train Size: 55119
2026-01-15 11:21:51,553 - logger - INFO - Val Size: 11812
2026-01-15 11:21:51,553 - logger - INFO - Test Size: 11812
2026-01-15 11:21:51,556 - logger - INFO - Class Distribution - Train: 3.88% pos, Val: 3.97% pos, Test: 4.10% pos
2026-01-15 11:21:51,582 - logger - INFO - Parameters: 124,586
2026-01-15 11:21:51,706 - logger - INFO - Using Asymmetric Loss (gamma_neg=4, gamma_pos=0, pos_weight=15)
2026-

Training finished.
